In [3]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/'My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive/My Drive


In [4]:
!pip install jsonlines

In [0]:
import json
import jsonlines
import sys
import re
sys.path.append('CSE576/data/')

In [0]:
file = "CSE576/data/Output/bert_train.json"

In [0]:
"""
with jsonlines.open(file, 'r') as f:
    data = f.read()
"""
#file = "CSE576/data/train1.0.json"
with open(file, "r") as read_file:
    data = json.load(read_file)

In [18]:
len(data)

2

In [19]:

data.keys()

dict_keys(['data', 'version'])

In [20]:
data['data'][0].keys()
len(data['data'])

10638

In [0]:
entry = data['data'][0]
paragraph = entry['document']
context_text = paragraph['context']
paragraph['qas']

In [0]:
doc = data['data'][0]['document']
context = doc['context']
"""
for qa in doc['qas']:
  print(qa['answers'])
"""
#print(context)

"\nfor qa in doc['qas']:\n  print(qa['answers'])\n"

In [0]:
def modify_qas(_qas):       # considers only the answer for the query present in the dataset
  for qs in range(0,len(_qas)):
    _qas[qs]['answers'] = _qas[qs]['answers'][0]
  return _qas

In [0]:
def modify_context(_data):        # removes the BEG__, __END and @placeholder tags
  rep = {'BEG__' : '', '__END' : '', '@placeholder': ''}
  rep = dict((re.escape(k), v) for k, v in rep.items()) 
  pattern = re.compile("|".join(rep.keys()))
  return pattern.sub(lambda m: rep[re.escape(m.group(0))], _data)

In [0]:
def modify_query(_qas):     # modifies the query by removing the BEG__, __END and @placeholder tags
  t = _qas
  for qs in range(0,len(t)):
    t[qs]['query'] = modify_context(t[qs]['query'])
  return t

In [0]:
for doc in range(0,len(data['data'])):
  temp = data['data'][doc]['document']['context']
  data['data'][doc]['document']['context'] = modify_context(temp)
  data['data'][doc]['document']['qas'] = modify_query(data['data'][doc]['document']['qas'])
  data['data'][doc]['document']['qas'] = modify_qas(data['data'][doc]['document']['qas'])
  # print(data['data'][doc]['document'])
# print(len(data['data']))

In [0]:
with open("CSE576/data/Output/bert_test.json",'w') as outfile:      // writes the data to the output file
  json.dump(data,outfile)

In [8]:
file = 'CSE576/data/Output_updated/bert_dev.json'
with jsonlines.open(file, 'r') as f:
    data = f.read()
print(len(data['data']))

584


In [0]:
import re

def search(text, context):    # searches for the 'answer' text in the context
  temp = context.find(text)
  return temp
count = 0

In [26]:
print(data['data'][1]['document'])
for _doc in range(0, len(data['data'])):    # adds the 'is_impossible' key to each query to bring the data into squad format based on whether we can find the answer text in context or not
  doc = data['data'][_doc]['document']
  _context = doc['context']
  _qas = doc['qas']

  for _qa in _qas:
    temp = search(_qa['answers']['text'], _context)
    total_count = total_count + 1
    if(temp != -1):
      _qa['answers']['answer_start'] = temp     # adds the start character of the answer that can be found in the context
      _qa['is_impossible'] = False

    elif(temp == -1):
      count = count + 1
      _qa['is_impossible'] = True
      #_qa['is_impossible'] = True

  temp_list = []
  for _qa in _qas:
    if(_qa['is_impossible'] == False):
      temp_list.append(_qa)
  data['data'][_doc]['document']['qas'] = temp_list

{'qas': [{'answers': {'text': 'spontaneous rupture', 'origin': 'dataset', 'sem_type': 'problem', 'cui': 'C0035956', 'answer_start': 1200}, 'id': 'bcr.06.2008.0080.5', 'query': 'Larger pseudoaneurysms have more chance of complications such as  , which may be fatal .', 'is_impossible': False}, {'answers': {'text': 'pseudoaneurysm of the radial artery', 'origin': 'dataset', 'sem_type': 'problem', 'cui': 'C0748224', 'answer_start': 332}, 'id': 'bcr.06.2008.0080.7', 'query': 'Even though ligation is an established procedure for  , vascular reconstruction is to be preferred if distal vascularity is clinically jeopardised or not documented with investigative modalities .', 'is_impossible': False}], 'title': 'BEG__Ruptured pseudoaneurysm of the radial artery__END', 'context': 'Summary\n\nA 40 - year - old man presented with a progressively increasing swelling over the ventral aspect of the right wrist of 23 days duration following a laceration .\nThe swelling was soft , pulsatile and was inter

In [0]:
with open("CSE576/data/Output_updated/bert_test.json",'w') as outfile:    # writes the converted data into the file
  json.dump(data,outfile)

In [0]:
file = "CSE576/data/Output_updated/bert_train.json"
with open(file, "r") as read_file:
    data = json.load(read_file)

In [13]:
      # this block of code checks for how many queries we have an answer with key as 'dataset'
count = 0
total_count = 0
for _document in range(0,len(data['data'])):
  document = data['data'][_document]['document']
  for _qa in range(0,len(document['qas'])):
    qas = document['qas'][_qa]
    # for _qaw in qas['answers']:
    total_count = total_count + 1
    
    if(qas['answers']['origin'] == 'dataset'):
        count = count+1
    """
    if(qas['is_impossible'] == True):
      count = count+1
    """
        #print(_qa)
print(count, total_count)   # prints the queries with key as 'dataset' and total  number of queries for each set(training,validation and test) based on what we load in the previous block

51061 51061


In [0]:
paragraph = data['data'][0]['document']   # this prints the answer along with the start character of the answer text
for qa in paragraph['qas']:
  answer = qa['answers']
  answer_text = answer["text"]
  start_position_character = answer["answer_start"]
  print(answer, answer_text, start_position_character)

{'text': 'amniotic band disruption', 'origin': 'dataset', 'sem_type': 'problem', 'cui': 'C1527388', 'answer_start': 151} amniotic band disruption 151
{'text': 'MRI', 'origin': 'dataset', 'sem_type': 'test', 'cui': 'C0024485', 'answer_start': 225} MRI 225


In [0]:
with open("CSE576/data/Output_updated/bert_test.json",'w') as outfile:
  json.dump(data,outfile)

In [0]:
temp_list = []
for _qa in _qas:
  if(_qa['is_impossible'] == False):
    temp_list.append(_qa)
temp_list


[{'answers': {'answer_start': 151,
   'cui': 'C1527388',
   'origin': 'dataset',
   'sem_type': 'problem',
   'text': 'amniotic band disruption'},
  'id': 'bcr.12.2009.2549.2',
  'is_impossible': False,
  'query': '▶ Isolated calvarial deformity mimicking caput succedenum from  is a possibility .'},
 {'answers': {'answer_start': 225,
   'cui': 'C0024485',
   'origin': 'dataset',
   'sem_type': 'test',
   'text': 'MRI'},
  'id': 'bcr.12.2009.2549.3',
  'is_impossible': False,
  'query': '▶ Careful neurological assessment and  are useful to exclude any neurological abnormality .'}]

In [0]:
ans = data['data'][1]['document']['qas'][0]['answers']
ans['origin']


In [0]:
_qas = data['data'][1]['document']['qas']
_qas

[{'answers': [{'cui': 'C0340652',
    'origin': 'dataset',
    'sem_type': 'problem',
    'text': 'Arterial injuries'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'arteries injury'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Arterial Injury'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'artery; injury'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Injury of artery (disorder)'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'arterial injuries'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Artery injury'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Injury of artery, NOS'},
   {'cui': 'C0340652',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'artery injuries'},
   {'cui': 'C0340652',
    'ori

In [0]:
with open(
            'CSE576/data/Output/bert_train.json', "r", encoding="utf-8"
        ) as reader:
        input_data = json.load(reader)["data"]
for entry in input_data:
  title = entry['document']['qas']
  for qa in title:
    answer = qa['answers']
    print(answer)
    break
  break

{'text': 'Isolated calvarial deformity mimicking caput succedenum', 'origin': 'dataset', 'sem_type': 'problem', 'cui': 'C2825501'}


In [0]:
_context = data['data'][1]['document']['context']
_context


'Summary\n\nA 40 - year - old man presented with a BEG__progressively increasing swelling__END over the ventral aspect of the right wrist of 23 days duration following a BEG__laceration__END .\nThe BEG__swelling__END was soft , pulsatile and was BEG__intermittently bleeding__END , with BEG__poor distal capillary filling__END and BEG__cold distal extremity__END .\nClinically it was diagnosed as BEG__pseudoaneurysm of the radial artery__END and was sent for a BEG__duplex scan__END for confirmation .\nThe BEG__lesion ruptured__END in the radiology department before the BEG__scan__END was done , and the patient went into BEG__hypovolaemia__END .\nIn view of this , the patient underwent BEG__exploration__END of the BEG__pseudoaneurysm under brachial block__END , and the radial artery was found to be transected just proximal to the dorsal branch .\nIt was decided to repair the artery as there was no evidence of adequate circulation supply or the patency of the ulnar artery preoperatively .\n

In [0]:
file = "CSE576/data/train1.0.json"
with open(file, "r") as read_file:
    data = json.load(read_file)

_qas = data['data'][0]['document']['qas']
_qas

[{'answers': [{'cui': 'C2825501',
    'origin': 'dataset',
    'sem_type': 'problem',
    'text': 'Isolated calvarial deformity mimicking caput succedenum'},
   {'cui': 'C2825501',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Calvarial'}],
  'id': 'bcr.12.2009.2549.1',
  'query': '▶ @placeholder from BEG__amniotic band disruption__END is a possibility .'},
 {'answers': [{'cui': 'C1527388',
    'origin': 'dataset',
    'sem_type': 'problem',
    'text': 'amniotic band disruption'},
   {'cui': 'C1527388',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Amniotic bands'},
   {'cui': 'C1527388',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Amniotic Band'},
   {'cui': 'C1527388',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'AMNIOTIC BAND'},
   {'cui': 'C1527388',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'Bands, Amniotic'},
   {'cui': 'C1527388',
    'origin': 'UMLS',
    'sem_type': 'problem',
    'text': 'a

'Summary\n\nA 40 - year - old man presented with a BEG__progressively increasing swelling__END over the ventral aspect of the right wrist of 23 days duration following a BEG__laceration__END .\nThe BEG__swelling__END was soft , pulsatile and was BEG__intermittently bleeding__END , with BEG__poor distal capillary filling__END and BEG__cold distal extremity__END .\nClinically it was diagnosed as BEG__pseudoaneurysm of the radial artery__END and was sent for a BEG__duplex scan__END for confirmation .\nThe BEG__lesion ruptured__END in the radiology department before the BEG__scan__END was done , and the patient went into BEG__hypovolaemia__END .\nIn view of this , the patient underwent BEG__exploration__END of the BEG__pseudoaneurysm under brachial block__END , and the radial artery was found to be transected just proximal to the dorsal branch .\nIt was decided to repair the artery as there was no evidence of adequate circulation supply or the patency of the ulnar artery preoperatively .\nBEG__Doppler study__END at 15 and 45 days postoperatively showed good flow through , with no evidence of BEG__thrombus__END .\n\nBACKGROUND\n\nBEG__Pseudoaneurysms occur rarely in the radial artery__END .\nAlmost all the cases reported to date have been secondary to BEG__radial artery cannulation__END or BEG__puncture__END for various indications .\nThere have been very few reported cases of BEG__spontaneous rupture of false aneurysm of the radial artery__END .\nWe report a case of a BEG__pseudoaneurysm of the radial artery__END occurring secondary to a BEG__laceration over the right wrist__END sustained while working in the garden ; the BEG__pseudoaneurysm__END ruptured spontaneously .\nBEG__False aneurysm of the radial artery__END , other than that due to a BEG__direct iatrogenic puncture__END or associated with BEG__fracture of bones__END , is not thought to have been reported in the literature .\nBEG__Spontaneous rupture of these lesions__END has been reported extremely rarely in the literature .\n\nCASE PRESENTATION\n\nA 40 - year - old man presented to us with a history of a soft , expanding , BEG__pulsatile mass over the right wrist__END , of 23 days duration .\nHe had suffered from BEG__laceration over the wrist__END around a month earlier , and this had been sutured at a peripheral hospital .\nHe noticed the BEG__swelling__END around a week later and it continued to increase in size , with BEG__progressive difficulty in extension of the wrist__END .\nHe started to BEG__bleed from the lesion__END intermittently after the BEG__sutures__END were removed .\nHe consulted the local hospital and was treated conservatively .\nAt this time he decided to come to us .\n\nOn BEG__examination__END , a soft , globular , BEG__pulsatile mass__END of 5 × 4 cm ( fig 1 ) with BEG__eroded skin__END in some areas was noted over the right wrist on the ventral aspect .\nThe wrist was in a slightly flexed and ulnar deviated position .\nA “ T ” BEG__shaped scar__END with BEG__suture marks__END could also be made out .\nBEG__Proximal pressure__END over the radial artery made the BEG__swelling non-pulsatile__END .\nBEG__Ulnar artery pulsation__END could not be felt .\nBEG__Capillary filling time__END was BEG__sluggish in right finger tips__END in comparison with the left , and the hand felt BEG__cold__END even though finger movements were present .\nA provisional diagnosis of BEG__false aneurysm of the radial artery__END was made and the patient was sent to the radiology department for a BEG__duplex scan__END .\n\nThe BEG__false aneurysm__END ruptured in the radiology department before a BEG__duplex scan__END could be performed , and the patient went into BEG__hypovolaemia__END due to BEG__bleeding__END .\nWe decided to operate and explore in view of the BEG__rupture__END , without the BEG__duplex scan__END report ( fig 2 ) .\n\nBEG__TREATMENT__END\n\nBEG__Exploration__END was done under brachial block with back - up for general anaesthesia , with a BEG__pneumatic tourniquet__END .\nThe proximal radial artery was exposed by BEG__vertical incision over the lower part of forearm__END , and skeletonised .\nThe artery distal to the BEG__aneurysm__END was also exposed and skeletonised .\nThe BEG__aneurysm sac__END was excised and BEG__clots__END removed , and this revealed transaction of the radial artery just proximal to the dorsal branch at the wrist .\nBEG__Profuse bleeding__END was noted from both ends on release of the tourniquet .\nProximal and distal vascular control was achieved by bulldog vascular clamps after heparinising both ends of the transected artery .\nThe radial artery was repaired using 6 / 0 prolene with BEG__interrupted sutures__END , and flow was restored on BEG__release of the bulldog vascular clamps__END ( fig 3 ) .\nThe skin was repaired with interrupted 1 / 0 BEG__prolene sutures__END .\nThe BEG__aneurysm sac__END was sent for BEG__culture and sensitivity__END .\n\nOUTCOME AND FOLLOW - UP\n\nThe patient was continued on BEG__heparin injections__END for 2 days postoperatively , and discharged on BEG__oral warfarin__END on the fifth postoperative day ( POD ) . BEG__Sutures__END were removed on 10th POD .\nA BEG__Doppler study__END on 15th POD showed BEG__weak flow through the anastomosis__END and a BEG__patent small ulnar artery__END .\nThe BEG__capillary filling__END improved and the BEG__temperature__END of both hands was equal .\nWrist and finger movements were BEG__pain__END free without any restrictions .\nBEG__Repeat Doppler study__END at 2 weeks and at 1.5 month follow - up showed good flow through the radial artery and maintenance of patency .\n\nDISCUSSION\n\nA BEG__false aneurysm__END is usually described as a BEG__pulsating haematoma__END that communicates with an artery through a BEG__disruption in the arterial wall__END .\nThere is no BEG__fixed definition but false aneurysms__END are irreversible and at least 50 % larger than the normal arterial diameter .\n1 The radial artery is most affected at the distal part where it lies superficially .\nBEG__Traumatic pseudoaneurysms__END may occur acutely or , more commonly , they may be discovered following BEG__unrecognised arterial injury .\nMany false aneurysms of the radial artery__END have been reported recently and most often they are localised in an area of BEG__arterial puncture__END or BEG__cannulation__END .\nOne - third of cases in a BEG__series__END presented with BEG__rupture__END requiring urgent opertion .\nControversy exists regarding the BEG__treatment alternatives__END for BEG__pseudoaneurysm of radial artery__END , and BEG__reconstruction__END with BEG__interposition graft__END has been advocated .\nBEG__Ligation__END of the artery is an alternative to BEG__end - to - end anastomosis__END in situations where there is a dual root of supply ( eg , in wrist and leg ) , demonstration of good collaterals in BEG__angiography__END , and excellent back BEG__bleeding from distal stump__END .\n6 A BEG__compression bandage__END , especially for children , offers an alternative to BEG__surgical procedure__END .\nBEG__Pseudoaneurysms__END less than 2 cm in diameter have a 70 % likelihood of BEG__spontaneous thrombosis__END with BEG__compression therapy__END , whereas larger ones and those in anticoagulated patients are likely to persist .\nBEG__Percutaneous thrombin injection__END has become an effective and alternative treatment option since the last decade .\nBEG__Radial artery pseudoaneurysm__END can also occur following missed BEG__traumatic lacerations__END other than BEG__iatrogenic arterial puncture__END .\nWe decided to reconstruct the artery by achieving BEG__end - to - end anastomosis__END , as distal vascularity was jeopardised and we did not have BEG__documentary evidence of circulation through the ulnar artery__END .\nWe believe that BEG__reconstruction__END should be attempted , especially in situations where distal vascularity is in doubt , even in areas where there are BEG__dual roots of circulation and adequate back bleeding from the distal stump__END .'

In [0]:
file = 'CSE576/data/Output_updated/bert_dev.json'
with jsonlines.open(file, 'r') as f:
    data = f.read()

file = "CSE576/data/Output_updated/gold_answers_dev_clicr.txt"
f = open(file, 'w')
count = 0

In [0]:
for _doc in range(0, len(data['data'])):
  doc = data['data'][_doc]['document']
  #_context = doc['context']
  _qas = doc['qas']

  for _qa in _qas:
    count = count + 1
    f.write(_qa['id'])
    f.write('\t')
    f.write( _qa['answers']['text'])
    f.write('\n')

    if(count == 500) : break
  if(count == 500): break

f.close()
f = open(file,'r')
len(f.readlines())


500